In [1]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)
import time
import warnings
warnings.filterwarnings("ignore")

import torch
random_seed = 42
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('./data/heloc_dataset.csv')
df['RiskPerformance']=(df.RiskPerformance=='Bad')+0

scaler = MinMaxScaler((-1,1))
X = scaler.fit_transform(df.values[:,1:])
y = df['RiskPerformance'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
df

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,MaxDelqEver,NumTotalTrades,NumTradesOpeninLast12M,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,1,55,144,4,84,20,3,0,83,2,3,5,23,1,43,0,0,0,33,-8,8,1,1,69
1,1,61,58,15,41,2,4,4,100,-7,0,8,7,0,67,0,0,0,0,-8,0,-8,-8,0
2,1,67,66,5,24,9,0,0,100,-7,7,8,9,4,44,0,4,4,53,66,4,2,1,86
3,1,66,169,1,73,28,1,1,93,76,6,6,30,3,57,0,5,4,72,83,6,4,3,91
4,1,81,333,27,132,12,0,0,100,-7,7,8,12,0,25,0,1,1,51,89,3,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10454,0,73,131,5,57,21,0,0,95,80,6,6,21,5,19,7,0,0,26,-8,5,2,0,100
10455,1,65,147,39,68,11,0,0,92,28,6,6,12,0,42,1,1,1,86,53,2,2,1,80
10456,1,74,129,6,64,18,1,1,100,-7,6,8,18,1,33,3,4,4,6,-8,5,-8,0,56
10457,1,72,234,12,113,42,2,2,96,35,6,2,45,0,20,6,0,0,19,-8,4,1,0,38


# BlackBoxes

### XGBOOST

In [4]:
from xgboost import XGBClassifier

clf_xgb = XGBClassifier(n_estimators=60, reg_lambda=3, use_label_encoder=False, eval_metric='logloss')

In [5]:
#clf_xgb.fit(X_train, Y_train)
#clf_xgb.save_model('./BlackBoxes/fico_xgboost')

In [6]:
clf_xgb.load_model('./BlackBoxes/fico_xgboost')
y_train_pred = clf_xgb.predict(X_train)
y_test_pred = clf_xgb.predict(X_test)
print('train acc:',np.mean(np.round(y_train_pred)==Y_train))
print('test acc:',np.mean(np.round(y_test_pred)==Y_test))


train acc: 0.8822605965463108
test acc: 0.7045191193511008


### Random Forest

In [7]:
#from sklearn.ensemble import RandomForestClassifier

#clf_rf = RandomForestClassifier(random_state=random_seed)
#clf_rf.fit(X_train, Y_train)

#pickle.dump(clf_rf,open('./BlackBoxes/fico_rf.p','wb'))

In [8]:
clf_rf = pickle.load(open('./BlackBoxes/fico_rf.p','rb'))
y_train_pred = clf_rf.predict(X_train)
y_test_pred = clf_rf.predict(X_test)
print('train acc:',np.mean(np.round(y_train_pred)==Y_train))
print('test acc:',np.mean(np.round(y_test_pred)==Y_test))


train acc: 0.977022977022977
test acc: 0.7117612977983777


### SVC

In [9]:
#from sklearn.svm import SVC
#clf_svc = SVC(gamma='auto', probability=True)
#clf_svc.fit(X_train, Y_train)

#pickle.dump(clf_svc,open('./BlackBoxes/fico_svc.p','wb'))

In [10]:
clf_svc = pickle.load(open('./BlackBoxes/fico_svc.p','rb'))
y_train_pred = clf_svc.predict(X_train)
y_test_pred = clf_svc.predict(X_test)
print('train acc:',np.mean(np.round(y_train_pred)==Y_train))
print('test acc:',np.mean(np.round(y_test_pred)==Y_test))

train acc: 0.7285571571285857
test acc: 0.7111819235225956


### NN tf

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

BATCH_SIZE = 1024
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(2048).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(BATCH_SIZE)

clf_nn = keras.Sequential([
    keras.layers.Dense(units=10, activation='relu'),
    keras.layers.Dense(units=5, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid'),
])

early_stopping = EarlyStopping(patience=5)

clf_nn.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

#history = clf_nn.fit(
#    train_dataset,
#    validation_data=test_dataset,
#    epochs=500,
#    callbacks=[early_stopping],
#    verbose=0
#)

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.grid()
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

#plot_metric(history, 'loss')
#clf_nn.save_weights('./blackboxes/fico_tf_nn')

In [12]:
from sklearn.metrics import accuracy_score
clf_nn.load_weights('./blackboxes/fico_tf_nn')
clf_nn.trainable = False
print(accuracy_score(np.round(clf_nn.predict(X_train)),Y_train))
print(accuracy_score(np.round(clf_nn.predict(X_test)),Y_test))

0.7278435849864421
0.7103128621089224


2022-03-24 11:07:47.561352: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-24 11:07:47.574526: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:258] None of the MLIR optimization passes are enabled (registered 0 passes)


### NN pt

In [13]:
#import torch.nn as nn
#import torch.nn.functional as F
#from torch.utils.data import TensorDataset, DataLoader
#import pytorch_lightning as pl
#
#class DataModule(pl.LightningDataModule):
#    def __init__(self, batch_size, X_train, Y_train, X_test, Y_test):
#        self.batch_size = batch_size
#        self.X_train = X_train
#        self.Y_train = Y_train
#        self.X_test = X_test
#        self.Y_test = Y_test
#
#    def train_dataloader(self):
#        train_dataset = TensorDataset(torch.tensor(self.X_train).float(),torch.tensor(self.Y_train,dtype=torch.long).float())
#        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True) 
#
#    def val_dataloader(self):
#        val_dataset = TensorDataset(torch.tensor(self.X_test).float(),torch.tensor(self.Y_test,dtype=torch.long).float())
#        return DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False) 
#
#class Lit_Dense_Network(pl.LightningModule):
#    def __init__(self):
#        super().__init__()
#        self.fc1 = nn.Linear(23, 10)
#        self.b1 = nn.BatchNorm1d(10)
#        self.fc2 = nn.Linear(10, 5)
#        self.b2 = nn.BatchNorm1d(5)
#        self.fc3 = nn.Linear(5,1)
#
#    def forward(self, x):
#        if len(x.shape)==1:
#            x = torch.reshape(x,[1,x.shape[0]])
#        x = F.relu(self.fc1(x))
#        x = self.b1(x)
#        x = F.relu(self.fc2(x))
#        x = self.b2(x)
#        x = F.sigmoid(self.fc3(x))
#        return x
#        
#    def training_step(self, batch, batch_idx):
#        x, y = batch
#        y_pred = self.forward(x)
#        loss = F.binary_cross_entropy(y_pred, y)
#        self.log('train_loss', loss)
#        return loss
#
#    def validation_step(self, batch, batch_idx):
#        x, y = batch
#        y_pred = self.forward(x)
#        loss = F.binary_cross_entropy(y_pred, y)
#        self.log('val_loss', loss)
#        return loss
#
#    def configure_optimizers(self):
#        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#        return optimizer

In [14]:
#from pytorch_lightning.callbacks.early_stopping import EarlyStopping
#from pytorch_lightning.callbacks.progress import TQDMProgressBar
#from pytorch_lightning.loggers import TensorBoardLogger
#
#net = Lit_Dense_Network()
#BATCH_SIZE = 1024
#dm = DataModule(BATCH_SIZE, X_train, Y_train.reshape(-1,1), X_test, Y_test.reshape(-1,1))
#bar = TQDMProgressBar(refresh_rate=20)
#es = EarlyStopping(monitor="val_loss", mode="min", patience=5)
#logger = TensorBoardLogger("lightning_logs", name="FICO_model", version=1)
#trainer = pl.Trainer(callbacks=[bar, es], 
#                     max_epochs=100, 
#                     log_every_n_steps=10, 
#                     logger=logger, 
#                     enable_checkpointing=False)
#
#trainer.fit(net, dm)
##trainer.save_checkpoint("fico_black_box.ckpt")

In [15]:
#clf_pt = Lit_Dense_Network()
#clf_pt.eval()
#clf_pt = clf_pt.load_from_checkpoint(checkpoint_path="./BlackBoxes/fico_pt_nn.ckpt")

In [16]:
#from sklearn.metrics import accuracy_score
#
#print(accuracy_score(np.round(clf_pt(torch.tensor(X_train).float()).detach().numpy()),Y_train))
#print(accuracy_score(np.round(clf_pt(torch.tensor(X_test).float()).detach().numpy()),Y_test))

### Predict Functions

In [17]:
def predict_clf_xgboost(x, return_proba=False):
    if return_proba:
        return clf_xgb.predict_proba(x)[:,1].ravel()
    else: return clf_xgb.predict(x).ravel().ravel()

def predict_clf_rf(x, return_proba=False):
    if return_proba:
        return clf_rf.predict_proba(x)[:,1].ravel()
    else: return clf_rf.predict(x).ravel().ravel()

def predict_clf_svc(x, return_proba=False):
    if return_proba:
        return clf_svc.predict_proba(x)[:,1].ravel()
    else: return clf_svc.predict(x).ravel().ravel()

def predict_clf_nn(x, return_proba=False):
    if return_proba:
        return clf_nn.predict(x).ravel()
    else: return np.round(clf_nn.predict(x).ravel()).astype(int).ravel()

# Latent Space

In [18]:
y_test_pred = predict_clf_nn(X_test, return_proba=True)
y_train_pred = predict_clf_nn(X_train, return_proba=True)
X_train = np.hstack((X_train,y_train_pred.reshape(-1,1)))
X_test = np.hstack((X_test,y_test_pred.reshape(-1,1)))

In [23]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

latent_dim = 7
batch_size = 512
sigma = 1
max_epochs = 1000
early_stopping = 3
learning_rate = 1e-3

similarity_KLD = torch.nn.KLDivLoss(reduction='batchmean')

def compute_similarity(X, sigma, idx_cat=None, alpha=1):
    D_features = torch.cdist(X[:,:-1],X[:,:-1])
    D_class = torch.cdist(X[:,-1].reshape(-1,1),X[:,-1].reshape(-1,1))
    D = D_features + alpha * D_class
    M = torch.exp((-D**2)/(2*sigma**2))
    return M / (torch.ones([M.shape[0],M.shape[1]])*(torch.sum(M, axis = 0)-1)).transpose(0,1)

def loss_function(X, Z, sigma=1, idx_cat=None ):
    Sx = compute_similarity(X, sigma, idx_cat, alpha=2)
    Sz = compute_similarity(Z, sigma=1, alpha=2)
    loss = similarity_KLD(torch.log(Sx), Sz)
    return loss

class LinearModel(nn.Module):
    def __init__(self, input_shape, latent_dim=2):
        super(LinearModel, self).__init__()

        # encoding components
        self.fc1 = nn.Linear(input_shape, latent_dim)

    def encode(self, x):
        x = self.fc1(x)
        return x

    def forward(self, x):
        z = self.encode(x)
        return z

# Create Model
model = LinearModel(X_train.shape[1], latent_dim=latent_dim)

In [24]:
train_dataset = TensorDataset(torch.tensor(X_train).float())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 
test_dataset = TensorDataset(torch.tensor(X_test).float())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) 

def check_and_clear(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    else:
        os.system('rm -r ' + dir_name)
        os.mkdir(dir_name)

check_and_clear('./models/weights')

model_params = list(model.parameters())
optimizer = torch.optim.Adam(model_params, lr=learning_rate)

# record training process
epoch_train_losses = []
epoch_test_losses = []

#validation parameters
epoch = 1
best = np.inf

# progress bar
pbar = tqdm(bar_format="{postfix[0]} {postfix[1][value]:03d} {postfix[2]} {postfix[3][value]:.5f} {postfix[4]} {postfix[5][value]:.5f} {postfix[6]} {postfix[7][value]:d}",
            postfix=["Epoch:", {'value':0}, "Train Sim Loss", {'value':0}, "Test Sim Loss", {'value':0}, "Early Stopping", {"value":0}])

# start training
while epoch <= max_epochs:

    # ------- TRAIN ------- #
    # set model as training mode
    model.train()
    batch_loss = []

    for batch, (X_batch,) in enumerate(train_loader):
        optimizer.zero_grad()
        Z_batch = model(X_batch)  #
        loss  = loss_function(X_batch, Z_batch, sigma) 
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())

    # save result
    epoch_train_losses.append(np.mean(batch_loss))
    pbar.postfix[3]["value"] = np.mean(batch_loss)

    # -------- VALIDATION --------

    # set model as testing mode
    model.eval()
    batch_loss = []

    with torch.no_grad():
        for batch, (X_batch,) in enumerate(test_loader):
            Z_batch = model(X_batch)
            loss = loss_function(X_batch, Z_batch, sigma)
            batch_loss.append(loss.item())

    # save information
    epoch_test_losses.append(np.mean(batch_loss))
    pbar.postfix[5]["value"] = np.mean(batch_loss)
    pbar.postfix[1]["value"] = epoch

    if epoch_test_losses[-1] < best:
        wait = 0
        best = epoch_test_losses[-1]
        best_epoch = epoch
        torch.save(model.state_dict(), f'./models/weights/LinearTransparent.pt')
    else:
        wait += 1
    pbar.postfix[7]["value"] = wait
    if wait == early_stopping:
        break    
    epoch += 1
    pbar.update()

model.load_state_dict(torch.load(f'./models/weights/LinearTransparent.pt'))
with torch.no_grad():
    model.eval()
    Z_train = model(torch.tensor(X_train).float()).cpu().detach().numpy()
    Z_test = model(torch.tensor(X_test).float()).cpu().detach().numpy()

torch.save(model.state_dict(), f'./models/fico_nn_{latent_dim}.pt')

Epoch: 000 Train Sim Loss 0.00000 Test Sim Loss 0.00000 Early Stopping 0

In [25]:
model.load_state_dict(torch.load(f'./models/fico_nn_{latent_dim}.pt'))
with torch.no_grad():
    model.eval()
    Z_train = model(torch.tensor(X_train).float()).cpu().detach().numpy()
    Z_test = model(torch.tensor(X_test).float()).cpu().detach().numpy()

In [26]:
#import math
#
#p_in = Z_test[idx,:]
#m = -2
#p_fin = Z_test[idx,:]+m*y_contrib
#
#eps = 0.1
#delta = math.sqrt((eps**2)/2)
#
#rule_1 = (Z_train[:,0]-(p_in[0]+delta))/(p_fin[0]-p_in[0])<(Z_train[:,1]-(p_in[1]+delta))/(p_fin[1]-p_in[1])
#rule_2 = (Z_train[:,0]-(p_in[0]-delta))/(p_fin[0]-p_in[0])>(Z_train[:,1]-(p_in[1]-delta))/(p_fin[1]-p_in[1])
#rule_3 = Z_train[:,1]<Z_train[:,0]-p_in[0]+p_in[1]
#rule_4 = Z_train[:,1]>Z_train[:,0]-p_fin[0]+p_fin[1]
#rule = rule_1*rule_2*rule_3*rule_4
#neigh_eps_train = X_train[rule,:]
#
#plt.scatter(Z_train[rule,0], Z_train[rule,1], c=y_train_pred[rule], cmap='coolwarm')
#plt.plot(p_in[0], p_in[1], 'kX', markersize=20)
#plt.plot(p_fin[0], p_fin[1], 'mX', markersize=20)
#plt.grid()
#
#rule_1 = (Z_test[:,0]-(p_in[0]+delta))/(p_fin[0]-p_in[0])<(Z_test[:,1]-(p_in[1]+delta))/(p_fin[1]-p_in[1])
#rule_2 = (Z_test[:,0]-(p_in[0]-delta))/(p_fin[0]-p_in[0])>(Z_test[:,1]-(p_in[1]-delta))/(p_fin[1]-p_in[1])
#rule_3 = Z_test[:,1]<Z_test[:,0]-p_in[0]+p_in[1]
#rule_4 = Z_test[:,1]>Z_test[:,0]-p_fin[0]+p_fin[1]
#rule = rule_1*rule_2*rule_3*rule_4
#neigh_eps_test = X_test[rule,:]
#

In [27]:
#z_neigh_eps_train = model(torch.tensor(neigh_eps_train).float()).detach().numpy()

In [28]:
#cdist(p_in.reshape(1,-1), z_neigh_eps_train)

In [29]:
#cdist(p_in.reshape(1,-1), Z_test)

In [30]:
#from sklearn.tree import DecisionTreeClassifier
#from sklearn import tree
#
#clf = DecisionTreeClassifier(max_depth=5, min_samples_leaf=10)
#clf.fit(neigh_eps_train[:,:-1],np.round(neigh_eps_train[:,-1]))
#plt.figure(figsize=(30,5))
#print('train acc:', clf.score(neigh_eps_train[:,:-1], np.round(neigh_eps_train[:,-1])))
#print('test acc:', clf.score(neigh_eps_test[:,:-1], np.round(neigh_eps_test[:,-1])))
#tree.plot_tree(clf,filled=True,rounded=True);

In [31]:
#q = X_test[idx,:-1].copy()
#pd.DataFrame([q], columns = df.columns[1:])

In [32]:
#print('original:', predict(q.reshape(1,-1),return_proba=True))
#q_mod = q.copy()
#q_mod[0] = 0.7
#q_mod[4] = -0.4
#q_mod[22] = 0.4
#print('mod:', predict(q_mod.reshape(1,-1),return_proba=True))

In [33]:
#q_mod = np.append(q_mod,predict(q_mod.reshape(1,-1)))
#z_q_mod = model(torch.tensor(q_mod).float()).detach().numpy()
#plt.scatter(Z_train[:,0], Z_train[:,1], c=y_train_pred, cmap='coolwarm')
#plt.plot(Z_test[idx,0], Z_test[idx,1], 'kX', markersize=20)
#m = -pred-1
#plt.plot(Z_test[idx,0]+m*y_contrib[0], Z_test[idx,1]+m*y_contrib[1], 'mX', markersize=20)
#plt.plot(z_q_mod[0], z_q_mod[1], 'gX', markersize=20)
#plt.grid()

## Counter Exemplars

In [55]:
w = model.fc1.weight.detach().numpy()
b = model.fc1.bias.detach().numpy()
y_contrib = model.fc1.weight.detach().numpy()[:,-1]

In [111]:
def compute_cf(q, indexes, debug=False):
    q_pred = predict_clf_nn(q[:-1].reshape(1,-1),return_proba=True)
    q_cf = q.copy()
    q_cf_preds = []
    q_cf_preds.append(q_pred)
    if q_pred > 0.5:
        m = -0.1
    else:
        m = 0.1
    while np.round(q_pred) == np.round(q_cf_preds[-1]):
        v = np.array(model(torch.tensor(q_cf).float()).detach().numpy()+m*y_contrib)
        c_l = [v[l] - np.sum(q_cf*w[l,:]) - b[l] for l in range(latent_dim)]
        M = []
        for l in range(latent_dim):
            M.append([np.sum(w[k,indexes]*w[l,indexes]) for k in range(latent_dim)])
        M = np.vstack(M)
        lambda_k = np.linalg.solve(M, c_l)
        delta_i = [np.sum(lambda_k*w[:,i]) for i in indexes]
        q_cf[indexes] += delta_i
        q_cf = np.clip(q_cf,-1,1)
        q_cf_new_pred = float(predict_clf_nn(q_cf[:-1].reshape(1,-1),return_proba=True))
        if debug:
            print(q_cf_new_pred)
        if q_cf_new_pred in q_cf_preds:
            return q_cf
        q_cf_preds.append(q_cf_new_pred)
        q_cf[-1] = q_cf_preds[-1]


    return q_cf

In [112]:
idx = 3
q = X_test[idx,:].copy()
print(predict_clf_nn(q[:-1].reshape(1,-1),return_proba=True))
#for i in [1,2,21,22,23]:
#    for indexes in  list(combinations(range(23),i)):
#        print(indexes)
q_cf = compute_cf(q, list(range(23)), True)
#q_cf[-1] = predict_clf_nn(q_cf[:-1].reshape(1,-1),return_proba=True) 
#print(predict_clf_nn(q_cf[:-1].reshape(1,-1),return_proba=True))
#pd.DataFrame([q[:-1],q_cf[:-1]], columns = df.columns[1:])

[0.58730996]
0.5739080905914307
0.581221342086792
0.5896790027618408
0.6007351279258728
0.6145113110542297
0.6235249638557434
0.6288937926292419
0.6342309713363647
0.6108070611953735
0.4887007772922516


In [113]:
from itertools import combinations
from tqdm.notebook import tqdm

d_dist = []
d_count = []
d_impl = []
num = []
div_dist = []
div_count = []

for idx in tqdm(range(10)):
    q = X_test[idx,:].copy()
    q_pred = np.round(predict_clf_nn(q[:-1].reshape(1,-1),return_proba=True))
    q_cfs = []
    for i in [1,2,21,22,23]:
        for indexes in  list(combinations(range(23),i)):    
            q_cf = compute_cf(q, list(indexes))
            q_cf_pred = predict_clf_nn(q_cf[:-1].reshape(1,-1),return_proba=True)
            if q_pred:
                if q_cf_pred < 0.5:
                    q_cfs.append(q_cf)
            else:
                if q_cf_pred > 0.5:
                    q_cfs.append(q_cf)
    if len(q_cfs)<1:
        continue
    else:
        q_cfs = np.vstack(q_cfs)
        d_dist.append(np.min(cdist(q_cfs[:,:-1],q[:-1].reshape(1,-1))))
        d_impl.append(np.min(cdist(q_cfs[:,:-1],X_train[:,:-1])))
        #d_count.append(1/((q_cfs.shape[1]-1)*q_cfs.shape[0])*np.sum(q_cfs[:,:-1]!=q[:-1]))
        d_count.append(np.min(np.sum(q_cfs[:,:-1]!=q[:-1],axis=1)))
        num.append(len(q_cfs))
        div_dist.append(1/(q_cfs.shape[0]**2)*np.sum(cdist(q_cfs[:,:-1],q_cfs[:,:-1])))
        div_count.append(23/(q_cfs.shape[0]**2)*np.sum(cdist(q_cfs[:,:-1], q_cfs[:,:-1],metric='hamming')))



  0%|          | 0/10 [00:00<?, ?it/s]

In [114]:
print('d_dist: \t',    np.round(np.mean(d_dist),5),   np.round(np.std(d_dist),5))
print('d_count: \t',   np.round(np.mean(d_count),5),  np.round(np.std(d_count),5))
print('implicity: \t', np.round(np.mean(d_impl),5),   np.round(np.std(d_impl),5))
print('number: \t',    np.round(np.mean(num),5),'\t', np.round(np.std(num),5))
print('div_dist: \t',  np.round(np.mean(div_dist),5), np.round(np.std(div_dist),5))
print('div_count: \t', np.round(np.mean(div_count),5),np.round(np.std(div_count),5))

print('success_rate: \t', len(d_dist)/10)

d_dist: 	 0.68979 0.78896
d_count: 	 1.7 2.1
implicity: 	 0.60548 0.49881
number: 	 348.4 	 51.65888
div_dist: 	 0.89605 0.31375
div_count: 	 19.50946 2.4938
success_rate: 	 1.0


# Growing Spheres

In [99]:
from growingspheres import counterfactuals as cf

idx = 0
q = X_test[idx,:-1].reshape(1,-1).copy()
pred = int(predict_clf_nn(q))
print(pred)

CF = cf.CounterfactualExplanation(q, predict_clf_nn, method='GS')
CF.fit(n_in_layer=2000, first_radius=0.1, dicrease_radius=10, sparse=True, verbose=True)
q_cf_GS = CF.enemy
print(predict_clf_nn(q_cf_GS.reshape(1,-1)))

0
1 ennemies found in initial sphere. Zooming in...
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  50
Final radius:  (0.08919999999999999, 0.09099999999999998)
Final number of ennemies:  1
Feature selection...
Reduced 17 coordinates
[1]


In [100]:
pd.DataFrame(np.vstack([q,q_cf_GS.reshape(1,-1)]), columns = df.columns[:-1])

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,MaxDelqEver,NumTotalTrades,NumTradesOpeninLast12M,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization
0,0.514563,-0.6133,-0.933673,-0.617347,0.136364,-0.357143,-0.357143,1.000000,-0.956522,0.777778,1.0,-0.115044,-0.071429,-0.651376,-0.878788,-0.733333,-0.733333,-0.659751,-0.7125,-0.219512,-0.25,-0.259259,0.082569
1,0.483616,-0.6133,-0.933673,-0.637759,0.101557,-0.357143,-0.357143,0.980184,-0.956522,0.777778,1.0,-0.115044,-0.071429,-0.651376,-0.878788,-0.683572,-0.733333,-0.640371,-0.7125,-0.219512,-0.25,-0.259259,0.082569


In [108]:
print('d_dist:',np.min(cdist(q_cf_GS.reshape(1,-1),q.reshape(1,-1))))
print('d_count:',1/(q_cf_GS.shape[0])*np.sum(q_cf_GS!=q))
print('impl:',np.min(cdist(q_cf_GS.reshape(1,-1),X_train[:,:-1])))

d_dist: 0.07635681600610537
d_count: 0.2608695652173913
impl: 0.4730768272887335


In [110]:
d_dist_GS = []
d_count_GS = []
d_impl_GS = []

for idx in tqdm(range(10)):
    q = X_test[idx,:-1].reshape(1,-1).copy()
    CF = cf.CounterfactualExplanation(q, predict_clf_nn, method='GS')
    CF.fit(n_in_layer=2000, first_radius=0.1, dicrease_radius=10, sparse=True, verbose=False)
    q_cf_GS = CF.enemy
    d_dist_GS.append(euclidean(q_cf_GS,q))
    d_count.append(1/(q_cf_GS.shape[0])*np.sum(q_cf_GS!=q))
    d_impl_GS.append(np.min(cdist(q_cf_GS.reshape(1,-1),X_train[:,:-1])))

  0%|          | 0/10 [00:00<?, ?it/s]

In [111]:
print('d_dist: \t',    np.round(np.mean(d_dist),5),   np.round(np.std(d_dist),5))
print('d_count: \t',   np.round(np.mean(d_count),5),  np.round(np.std(d_count),5))
print('implicity: \t', np.round(np.mean(d_impl),5),   np.round(np.std(d_impl),5))

d_dist: 	 0.44098 0.28471
d_count: 	 0.5448 0.14465
implicity: 	 0.47295 0.17419


# DICE

In [18]:
import dice_ml

dataset = pd.DataFrame(np.hstack((Y_train.reshape(-1,1),X_train)), columns=list(df.columns))
d = dice_ml.Data(dataframe=dataset, continuous_features=list(dataset.columns[1:]), outcome_name='RiskPerformance')

In [19]:
dataset

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,MaxDelqEver,NumTotalTrades,NumTradesOpeninLast12M,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,1.0,0.533981,-0.600985,-0.918367,-0.551020,-0.068182,-0.357143,-0.357143,0.798165,-0.434783,0.555556,0.764706,-0.203540,-0.142857,-0.174312,-0.454545,-0.706667,-0.733333,-0.286307,-0.650000,-0.219512,-0.3750,-0.259259,0.798165
1,1.0,0.417476,-0.724138,-0.948980,-0.688776,-0.295455,-0.285714,-0.285714,0.779817,-0.239130,0.666667,0.764706,-0.415929,-0.142857,-0.137615,-0.393939,-0.626667,-0.626667,-0.684647,-0.641667,-0.365854,-0.2500,-0.185185,0.229358
2,1.0,0.786408,-0.460591,-0.938776,-0.576531,-0.250000,-0.357143,-0.357143,1.000000,-0.956522,0.777778,1.000000,-0.415929,-0.142857,-0.302752,-0.454545,-0.680000,-0.680000,-0.834025,-0.600000,-0.463415,-0.3125,-0.333333,-0.100917
3,0.0,0.708738,-0.633005,-0.928571,-0.617347,-0.500000,-0.357143,-0.357143,1.000000,-0.956522,0.777778,1.000000,-0.610619,-0.285714,-0.412844,0.636364,-0.760000,-0.760000,-0.618257,-0.579167,-0.512195,-0.3125,-0.259259,0.633028
4,1.0,0.864078,-0.549261,-0.943878,-0.653061,-0.431818,-0.357143,-0.357143,1.000000,-0.956522,0.777778,1.000000,-0.504425,-0.071429,-0.449541,-0.454545,-0.706667,-0.706667,-0.842324,-0.558333,-0.268293,-0.3125,-0.333333,0.798165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002,1.0,0.436893,-0.517241,-0.923469,-0.515306,-0.090909,-0.357143,-0.357143,0.889908,-0.434783,0.666667,0.764706,-0.274336,-0.285714,-0.211009,-0.454545,-0.733333,-0.733333,-0.443983,-0.641667,-0.073171,-0.1875,-0.037037,0.871560
7003,0.0,0.728155,-0.635468,-0.948980,-0.750000,-0.522727,-0.357143,-0.357143,1.000000,-0.956522,0.777778,1.000000,-0.592920,-0.071429,-0.174312,-0.333333,-0.626667,-0.626667,-0.767635,-0.558333,-0.317073,-0.3125,-0.333333,0.779817
7004,0.0,0.864078,-0.411330,-0.948980,-0.530612,-0.295455,-0.357143,-0.357143,1.000000,-0.956522,0.777778,1.000000,-0.433628,-0.214286,-0.431193,-0.878788,-0.760000,-0.760000,-0.917012,-0.754167,-0.512195,-0.3125,-0.333333,-0.064220
7005,1.0,0.495146,-0.652709,-0.933673,-0.704082,-0.431818,-0.285714,-0.285714,0.889908,-0.391304,0.666667,0.764706,-0.557522,-0.285714,0.082569,-0.454545,-0.680000,-0.680000,-0.510373,-0.645833,-0.317073,-0.3125,-0.333333,0.779817


In [28]:
#m = dice_ml.Model(model=clf_nn, backend='TF2')
m = dice_ml.Model(model=clf_rf, backend='sklearn')
#clf_pt.eval()
#m = dice_ml.Model(model=clf_pt, backend='PYT')
# initiate DiCE
exp = dice_ml.Dice(d, m, method='random')

In [29]:
query_instances = dataset.iloc[0:1,1:]

# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instances, 
                                        total_CFs=4, 
                                        desired_class="opposite", 
                                        verbose=False)

100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


In [30]:
q_cfs_dice = dice_exp.cf_examples_list[0].final_cfs_df.values
q_cfs_dice.shape

(4, 24)

In [18]:
from scipy.spatial.distance import cdist

d_dist_dice = []
d_count_dice = []
d_impl_dice = []
num_dice = []
div_dist_dice = []
div_count_dice = []

for i in tqdm(range(10)):
    query_instances = dataset.iloc[i:i+1,1:]

    dice_exp = exp.generate_counterfactuals(query_instances, 
                                            total_CFs=4, 
                                            desired_class="opposite", 
                                            verbose=False)
    q_cfs_dice = dice_exp.cf_examples_list[0].final_cfs_df.values
    
    d_dist_dice.append(np.min(cdist(q_cfs_dice[:,:-1],query_instances.values)))
    d_count_dice.append(1/(q_cfs_dice.shape[1]*q_cfs_dice.shape[0])*np.sum(q_cfs_dice[:,:-1]!=query_instances.values))
    d_impl_dice.append(np.min(cdist(q_cfs_dice[:,:-1],X_train)))
    num_dice.append(len(q_cfs_dice))
    div_dist_dice.append(1/(q_cfs_dice.shape[0]**2)*np.sum(cdist(q_cfs_dice[:,:-1],q_cfs_dice[:,:-1])))
    div_count_dice.append(23/(q_cfs_dice.shape[0]**2)*np.sum(cdist(q_cfs_dice[:,:-1], q_cfs_dice[:,:-1],metric='hamming')))
    

  0%|          | 0/10 [00:00<?, ?it/s]

Diverse Counterfactuals found! total time taken: 00 min 22 sec


Diverse Counterfactuals found! total time taken: 01 min 36 sec


Diverse Counterfactuals found! total time taken: 00 min 45 sec


Diverse Counterfactuals found! total time taken: 00 min 30 sec


Diverse Counterfactuals found! total time taken: 00 min 44 sec


Diverse Counterfactuals found! total time taken: 00 min 23 sec


Diverse Counterfactuals found! total time taken: 00 min 41 sec


Diverse Counterfactuals found! total time taken: 00 min 20 sec


Diverse Counterfactuals found! total time taken: 00 min 20 sec
Diverse Counterfactuals found! total time taken: 00 min 25 sec


In [19]:
print('d_dist: \t',    np.round(np.mean(d_dist_dice),5),   np.round(np.std(d_dist_dice),5))
print('d_count: \t',   np.round(np.mean(d_count_dice),5),  np.round(np.std(d_count_dice),5))
print('implicity: \t', np.round(np.mean(d_impl_dice),5),   np.round(np.std(d_impl_dice),5))
print('number: \t',    np.round(np.mean(num_dice),5),'\t', np.round(np.std(num_dice),5))
print('div_dist: \t',  np.round(np.mean(div_dist_dice),5), np.round(np.std(div_dist_dice),5))
print('div_count: \t', np.round(np.mean(div_count_dice),5),np.round(np.std(div_count_dice),5))

print('success_rate: \t', len(d_dist_dice)/10)

d_dist: 	 2.20078 0.33072
d_count: 	 0.85417 0.20215
implicity: 	 1.74571 0.29195
number: 	 4.0 	 0.0
div_dist: 	 0.90989 0.23642
div_count: 	 11.075 2.09404
success_rate: 	 1.0


# Watcher (FAT-Forensic)

In [20]:
clf_nn.predict

<bound method Model.predict of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f8d0a854210>>

In [25]:
fat_model.predict(X_train)

array([1, 1, 0, ..., 0, 1, 1])

In [33]:
import fatf.utils.data.datasets as fatf_datasets
import fatf.utils.models as fatf_models

import fatf.transparency.predictions.counterfactuals as fatf_cf

class FAT_wrapper():
    def __init__(self,model):
        self.model = model

    def predict(self,x):
        return np.round(self.model.predict(x).ravel()).astype(int).ravel()

    def fit(self, x, y):
        pass

    def predict_proba(self,x):
        return self.model.predict(x).ravel()

fat_model = FAT_wrapper(clf_nn)

# Create a Counterfactual Explainer
cf_explainer = fatf_cf.CounterfactualExplainer(
    model=fat_model,
    dataset=X_train,
    categorical_indices=[],
    default_numerical_step_size=0.1)

In [38]:
dp_1_cf_tuple = cf_explainer.explain_instance(X_test[0])
dp_1_cfs, dp_1_cfs_distances, dp_1_cfs_predictions = dp_1_cf_tuple

In [39]:
clf_nn.predict(X_test[0:1])

array([[0.4551267]], dtype=float32)

In [40]:
dp_1_cfs_predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [42]:
dp_1_cfs.shape

(283, 23)

In [44]:
q.reshape(1,-1).shape

(1, 23)

# CEM

In [9]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from alibi.explainers import CEM

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.5.0
Eager execution enabled:  False


In [11]:
idx = 2
X = X_test[idx].reshape(1,-1)
print(f'Prediction probabilities for each class on the instance: {clf_nn.predict(X)}')

Prediction probabilities for each class on the instance: [[0.8737931]]


In [12]:
mode = 'PN'  # 'PN' (pertinent negative) or 'PP' (pertinent positive)
shape = (1,) + X_train.shape[1:]  # instance shape
kappa = .2  # minimum difference needed between the prediction probability for the perturbed instance on the
            # class predicted by the original instance and the max probability on the other classes
            # in order for the first loss term to be minimized
beta = .1  # weight of the L1 loss term
c_init = 5.  # initial weight c of the loss term encouraging to predict a different class (PN) or
              # the same class (PP) for the perturbed instance compared to the original instance to be explained
c_steps = 10  # nb of updates for c
max_iterations = 1000  # nb of iterations per value of c
feature_range = (X_train.min(axis=0).reshape(shape),  # feature range for the perturbed instance
                 X_train.max(axis=0).reshape(shape))  # can be either a float or array of shape (1xfeatures)
clip = (-1000.,1000.)  # gradient clipping
lr_init = 1e-2  # initial learning rate

In [13]:
# initialize CEM explainer and explain instance
cem = CEM(clf_nn, mode, shape, kappa=kappa, beta=beta, feature_range=feature_range,
          max_iterations=max_iterations, c_init=c_init, c_steps=c_steps,
          learning_rate_init=lr_init, clip=clip)
cem.fit(X_train, no_info_type='median')  # we need to define what feature values contain the least
                                         # info wrt predictions
                                         # here we will naively assume that the feature-wise median
                                         # contains no info; domain knowledge helps!
explanation = cem.explain(X, verbose=False)

No PN found!


In [27]:
mode = 'PP'

# initialize CEM explainer and explain instance
cem = CEM(clf_nn, mode, shape, kappa=kappa, beta=beta, feature_range=feature_range,
          max_iterations=max_iterations, c_init=c_init, c_steps=c_steps,
          learning_rate_init=lr_init, clip=clip)
cem.fit(X_train, no_info_type='median')
explanation = cem.explain(X, verbose=False)

In [32]:
pd.DataFrame(explanation.PP)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,-8.101602e-09,1.335966e-08,2.919411e-08,-1.094779e-08,8.127906e-09,8.514949e-09,8.514949e-09,0.0,1.036603e-08,-1.324548e-08,0.0,2.109899e-09,2.128737e-09,4.784777e-09,1.354651e-08,-9.536743e-09,-9.536743e-09,2.324828e-08,-1.192093e-08,8.722631e-09,0.0,-5.518949e-09,4.237945e-09


# Guided Prototypes

In [14]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from alibi.explainers import CounterfactualProto

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.5.0
Eager execution enabled:  False


In [17]:
idx = 0
X = X_test[idx].reshape((1,) + X_test[idx].shape)
shape = X.shape

In [22]:
# initialize explainer, fit and generate counterfactual
cf = CounterfactualProto(clf_nn, shape, use_kdtree=False, theta=10., max_iterations=1000,
                         feature_range=(X_train.min(axis=0), X_train.max(axis=0)),
                         c_init=1., c_steps=10)

cf.fit(X_train)
explanation = cf.explain(X)

No counterfactual found!


In [21]:
X_train.max(axis=0)

array([ 0.98058252,  1.        ,  1.        ,  1.        ,  1.        ,
        0.85714286,  0.78571429,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.4439834 , -0.14583333,  1.        ,
        0.75      ,  1.        ,  1.        ])